In [1]:
# Plotting notebook for the project
# Imports
import argparse
import os
import sys
import glob
import re

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from datetime import datetime
import scipy.stats as stats

# import cdo
from cdo import *
cdo = Cdo()

# Local imports
sys.path.append('/home/users/benhutch/skill-maps-historical')
import dictionaries as dic
import functions as fnc

In [2]:
# Set up the parameters again
# for the years 2-9 psl ULG (JJA)
var = "psl"
region = "global"
region_grid = dic.gridspec_global
forecast_range = "2-9"
season = "DJFM" # weird season name for model
observations_path = dic.obs
obs_var_name = "psl"
model_dict = dic.model_dictionary_psl_historical_badc
start_year = 1960
end_year = 2014

In [3]:
# # Process the observations first of all
# obs = fnc.process_observations(var, region, region_grid,
#                                forecast_range, season, observations_path,
#                                obs_var_name)

In [4]:
# First we want to merge the time axis and regrid the model (historical) data
# using the function call_mergetime_regrid
# this does not return anything, but saves the regridded data to a netcdf file
fnc.call_mergetime_regrid(model_dict, var, region)

processing model:  BCC-CSM2-MR
runs for model  BCC-CSM2-MR :  [1, 2, 3]
processing run:  1
processing init scheme:  1
processing physics scheme:  1
processing forcing scheme:  1
dir_path:  /badc/cmip6/data/CMIP6/CMIP/*/BCC-CSM2-MR/historical/r1i1p1f1/Amon/psl/g?/files/d????????/
Only one file available
File copied successfully
copied_file:  /gws/nopw/j04/canari/users/benhutch/historical/psl/BCC-CSM2-MR/mergetime/psl_Amon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.nc
Searching for merged file:  /gws/nopw/j04/canari/users/benhutch/historical/psl/BCC-CSM2-MR/mergetime/psl_Amon_BCC-CSM2-MR_historical_r1i1p1f1_g?_*.nc
type(merged_file):  <class 'list'>
merged_file:  ['/gws/nopw/j04/canari/users/benhutch/historical/psl/BCC-CSM2-MR/mergetime/psl_Amon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.nc']
merged_file[0]:  /gws/nopw/j04/canari/users/benhutch/historical/psl/BCC-CSM2-MR/mergetime/psl_Amon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.nc
Output file does not exist
Files re

AttributeError: 'list' object has no attribute 'split'

In [ ]:
# Now we want to load the historical data
# as a dictionary of xarray datasets for each model
# using the load_historical_data function
historical_data = fnc.load_historical_data(model_dict, var, region)

In [ ]:
# now we want to call the function to process the historical data
# this will:
# 1. constrain the data to the provided year range and season
# 2. calculate the climatology and remove this to create anomalies
# 3. calculate the annual mean anomalies from the monthly mean anomalies
# 4. calculate the running mean of these annual mean anomalies
# then add the processed data back into the dictionary
historical_data = fnc.process_historical_data(historical_data, season, forecast_range, start_year, end_year)

In [ ]:
# Now we want to process the historical data
# in preperation for calculating the spatial correlations
# this function constrains the years to only those available in all members
# and then calculates the equally weighted ensemble mean of all members
# using the function process_historical_data_spacial_correlations
ensemble_mean = fnc.process_historical_data_spatial_correlations(historical_data)